In [1]:
import sys, os

src = r"C:\Users\user\Desktop\Coding mo\AutoML proj1\task 1\AutoML Project Molham\src" # copy the location of the folder that has paths.py file using (ctrl + shift + C ) and paste it here    
sys.path.append(src)

from paths import *

get_paths()

sys.path.append(SRC_PATH)

from pipeline import *




In [ ]:
# ---------------- Task 1+9 ----------------

from flaml import AutoML

def loop_dfs_and_evaluate_flaml():
    for dataset_file in data_dict_classification_only.keys():
        dataset_summary = []
        runtime_memory_rows = []
        dataset_base = os.path.splitext(dataset_file)[0]

        for seed in RANDOM_SEEDS:
            np.random.seed(seed)
            random.seed(seed)

            df_train = splits_dict[dataset_file]['train']
            df_test  = splits_dict[dataset_file]['test']
            task_type = tasks_dict_classification_only[dataset_file]

            automl = AutoML()
            start_time = time.time()
            automl.fit(
                X_train=df_train.drop(columns=[TARGET_COLS[dataset_file]]),
                y_train=df_train[TARGET_COLS[dataset_file]],
                task=task_type,
                time_budget=TIME_BUDGET,
                eval_method='holdout',
                split_ratio=0.8,
                ensemble=True,
                seed=seed,
                estimator_list=["lgbm", "xgboost"],
                n_jobs=1,
                verbose=0 
            )
            training_runtime_sec = round(time.time() - start_time, 3)

            infer_start = time.time()
            results = evaluate_and_save_results_general(df_train, df_test, automl, dataset_file, framework='flaml')
            inference_time_per_sample = round((time.time() - infer_start) / len(df_test), 6)
            cpu_usage_percent = psutil.cpu_percent(interval=None)

            temp_model_file = os.path.join("temp_flaml_models", f"{dataset_base}_{seed}.pkl")
            os.makedirs("temp_flaml_models", exist_ok=True)
            with open(temp_model_file, "wb") as f:
                pickle.dump(automl, f)
                
            total_bytes = os.path.getsize(temp_model_file)
            model_size_mb = round(total_bytes / (1024*1024), 3)
            os.remove(temp_model_file)

            memory_usage_mb = get_memory_usage_mb()

            runtime_memory_rows.append({
                "dataset": dataset_file,
                "seed": seed,
                "training_runtime_sec": training_runtime_sec,
                "model_size_MB": model_size_mb,
                "memory_usage_MB": memory_usage_mb,
                "inference_time_per_sample_sec": inference_time_per_sample,
                "cpu_usage_percent": cpu_usage_percent
            })

            dataset_summary.append({
                'dataset': dataset_file,
                'seed': seed,
                **results,
                'runtime': training_runtime_sec
            })

            del automl
            gc.collect()

        output_path = os.path.join(AUTOML_METRICS_PATH, f"flaml_{dataset_base}_summary.csv")
        pd.DataFrame(dataset_summary).to_csv(output_path, index=False)

        task1_output_path = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"flaml_{dataset_base}_task1_runtime_memory_metrics.csv")
        pd.DataFrame(runtime_memory_rows).to_csv(task1_output_path, index=False)

        json_path = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"flaml_{dataset_base}_runtime_memory_metrics_in_json.json")
        with open(json_path, 'w') as f:
            json.dump(runtime_memory_rows, f, indent=2)

loop_dfs_and_evaluate_flaml()


In [3]:
for dataset_name in data_dict_classification_only.keys():
    dataset_base = dataset_name.replace('.csv', '')
    
    csv_file1 = os.path.join(AUTOML_METRICS_PATH, f"flaml_{dataset_base}_summary.csv")
    csv_file2 = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"flaml_{dataset_base}_task1_runtime_memory_metrics.csv")

    print(f"CSV for {dataset_name.replace('.csv', '')}: flaml_{dataset_base}_summary.csv")
    df1 = pd.read_csv(csv_file1)
    display(df1)

    print(f"CSV for {dataset_name.replace('.csv', '')}: flaml_{dataset_base}_task1_runtime_memory_metrics.csv")
    df2 = pd.read_csv(csv_file2)
    display(df2)



CSV for modeldata: flaml_modeldata_summary.csv


,dataset,seed,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,0.927417,0.927497,0.927602,0.927417,48.746
1,modeldata.csv,123,0.929638,0.929802,0.930076,0.929638,47.668


CSV for modeldata: flaml_modeldata_task1_runtime_memory_metrics.csv


,dataset,seed,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,modeldata.csv,42,48.746,0.341,7277.211,0.000023,15.1
1,modeldata.csv,123,47.668,0.412,7467.684,0.000021,14.6


CSV for titanic: flaml_titanic_summary.csv


...

,dataset,seed,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,train.csv,42,30.122,2.473,7257.176,0.000369,26.5
1,train.csv,123,30.113,2.172,7257.160,0.000301,31.3


CSV for wine: flaml_wine_summary.csv


,dataset,seed,accuracy,f1,precision,recall,runtime
0,wine.csv,42,1.0,1.0,1.0,1.0,30.315
1,wine.csv,123,1.0,1.0,1.0,1.0,30.340


CSV for wine: flaml_wine_task1_runtime_memory_metrics.csv


,dataset,seed,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,wine.csv,42,30.315,2.059,7257.285,0.000389,28.1
1,wine.csv,123,30.340,1.682,7257.375,0.000363,31.0


In [4]:
# ---------------- Task 3+9 ----------------

from flaml import AutoML

def evaluate_cv_folds_flaml(folds_dict):
    for dataset_file, folds_data in folds_dict.items():
        task_type = tasks_dict_classification_only[dataset_file]
        target_col = TARGET_COLS[dataset_file]

        dataset_summary = []
        runtime_memory_rows = []
        dataset_base = dataset_file.replace(".csv", "")

        for seed in RANDOM_SEEDS:
            np.random.seed(seed)
            random.seed(seed)

            fold_num = 1
            for fold_name, fold_splits in folds_data.items():
                df_train = fold_splits['train']
                df_val   = fold_splits['val']

                X_val = df_val.drop(columns=[target_col])
                y_val = df_val[target_col]

                temp_dir = tempfile.mkdtemp()

                start_time = time.time()
                automl = AutoML()
                automl.fit(
                    X_train=df_train.drop(columns=[target_col]),
                    y_train=df_train[target_col],
                    task=task_type,
                    time_budget=CV_TIME_BUDGET,
                    eval_method='holdout',
                    split_ratio=0.8,
                    ensemble=True,
                    seed=seed,
                    estimator_list=["lgbm", "xgboost"],
                    n_jobs=1,
                    verbose=0

                )
                training_runtime_sec = round(time.time() - start_time, 3)

                infer_start = time.time()
                results = evaluate_and_save_results_general(df_train, df_val, automl, dataset_file, framework='flaml')
                inference_time_per_sample = round((time.time() - infer_start) / len(X_val), 6)
                cpu_usage_percent = psutil.cpu_percent(interval=None)

                temp_model_file = os.path.join(temp_dir, f"{dataset_base}_{seed}_fold{fold_num}.pkl")
                os.makedirs("temp_flaml_models", exist_ok=True)
                with open(temp_model_file, "wb") as f:
                        pickle.dump(automl, f)
                    
                total_bytes = os.path.getsize(temp_model_file)
                model_size_mb = round(total_bytes / (1024*1024), 3)
                os.remove(temp_model_file)
                shutil.rmtree(temp_dir, ignore_errors=True)

                memory_usage_mb = get_memory_usage_mb()

                runtime_memory_rows.append({
                    "dataset": dataset_file,
                    "seed": seed,
                    "fold": fold_num,
                    "training_runtime_sec": training_runtime_sec,
                    "model_size_MB": model_size_mb,
                    "memory_usage_MB": memory_usage_mb,
                    "inference_time_per_sample_sec": inference_time_per_sample,
                    "cpu_usage_percent": cpu_usage_percent
                })

                dataset_summary.append({
                    'dataset': dataset_file,
                    'seed': seed,
                    'fold': fold_num,
                    **results,
                    'runtime': training_runtime_sec
                })

                fold_num += 1
                del automl
                gc.collect()

        output_path = os.path.join(CV_FOLDS_METRICS_PATH, f"flaml_{dataset_base}_cv_summary.csv")
        pd.DataFrame(dataset_summary).to_csv(output_path, index=False)

        task3_output_path = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"flaml_{dataset_base}_task3_runtime_memory_metrics.csv")
        pd.DataFrame(runtime_memory_rows).to_csv(task3_output_path, index=False)

        json_path = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"flaml_{dataset_base}_runtime_memory_metrics_cv_in_json.json")
        with open(json_path, 'w') as f:
            json.dump(runtime_memory_rows, f, indent=2)

evaluate_cv_folds_flaml(folds_dict)


In [5]:
for dataset_name in data_dict_classification_only.keys():
    dataset_base = dataset_name.replace('.csv', '')
    csv_file1 = os.path.join(CV_FOLDS_METRICS_PATH, f"flaml_{dataset_base}_cv_summary.csv")
    csv_file2 = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"flaml_{dataset_base}_task3_runtime_memory_metrics.csv")

    print(f"CSV for {dataset_base}: flaml_{dataset_base}_cv_summary.csv")
    df1 = pd.read_csv(csv_file1)
    display(df1)

    print(f"CSV for {dataset_base}: flaml_{dataset_base}_task3_runtime_memory_metrics.csv")
    df2 = pd.read_csv(csv_file2)
    display(df2)



CSV for modeldata: flaml_modeldata_cv_summary.csv


,dataset,seed,fold,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,1,0.809365,0.812267,0.821786,0.809365,22.880
1,modeldata.csv,42,2,0.791597,0.795682,0.815038,0.791597,22.950
2,modeldata.csv,42,3,0.790795,0.794766,0.812064,0.790795,22.842
3,modeldata.csv,42,4,0.800265,0.803590,0.815449,0.800265,23.135
4,modeldata.csv,42,5,0.819421,0.821728,0.828210,0.819421,23.381
5,modeldata.csv,123,1,0.783392,0.786907,0.798135,0.783392,22.541
6,modeldata.csv,123,2,0.765593,0.769965,0.786354,0.765593,22.536
7,modeldata.csv,123,3,0.783639,0.787414,0.800912,0.783639,23.241
8,modeldata.csv,123,4,0.776914,0.781093,0.797677,0.776914,23.118
9,modeldata.csv,123,5,0.777747,0.781764,0.796666,0.777747,22.560


CSV for modeldata: flaml_modeldata_task3_runtime_memory_metrics.csv


,dataset,seed,fold,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,modeldata.csv,42,1,22.880,0.249,7351.977,0.000020,15.9
1,modeldata.csv,42,2,22.950,0.255,7332.430,0.000021,14.3
2,modeldata.csv,42,3,22.842,0.249,7660.551,0.000018,14.4
3,modeldata.csv,42,4,23.135,0.252,7353.324,0.000021,14.3
4,modeldata.csv,42,5,23.381,0.257,7639.941,0.000020,14.7
5,modeldata.csv,123,1,22.541,0.219,7705.930,0.000018,14.7
6,modeldata.csv,123,2,22.536,0.317,7370.027,0.000021,14.4
7,modeldata.csv,123,3,23.241,0.316,7670.723,0.000020,13.5
8,modeldata.csv,123,4,23.118,0.321,7493.980,0.000022,14.0
9,modeldata.csv,123,5,22.560,0.205,7703.820,0.000020,13.9


CSV for titanic: flaml_titanic_cv_summary.csv


...

,dataset,seed,fold,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,train.csv,42,1,10.109,0.828,7697.152,0.000368,29.8
1,train.csv,42,2,10.097,0.879,7696.930,0.000303,26.8
2,train.csv,42,3,10.122,0.898,7697.145,0.000367,27.8
3,train.csv,42,4,10.098,0.771,7697.137,0.000367,32.8
4,train.csv,42,5,10.094,0.815,7697.148,0.000401,33.8
5,train.csv,123,1,10.116,0.742,7697.148,0.000368,31.0
6,train.csv,123,2,10.090,0.841,7697.137,0.000433,31.3
7,train.csv,123,3,10.110,0.759,7697.145,0.000367,31.8
8,train.csv,123,4,10.106,0.746,7697.133,0.000300,33.8
9,train.csv,123,5,10.111,0.729,7697.148,0.000302,33.9


CSV for wine: flaml_wine_cv_summary.csv


,dataset,seed,fold,accuracy,f1,precision,recall,runtime
0,wine.csv,42,1,0.944444,0.944502,0.946970,0.944444,10.163
1,wine.csv,42,2,1.000000,1.000000,1.000000,1.000000,10.155
2,wine.csv,42,3,0.916667,0.916529,0.922466,0.916667,10.123
3,wine.csv,42,4,0.971429,0.971300,0.973333,0.971429,10.145
4,wine.csv,42,5,1.000000,1.000000,1.000000,1.000000,10.148
5,wine.csv,123,1,0.944444,0.944502,0.946970,0.944444,10.154
6,wine.csv,123,2,1.000000,1.000000,1.000000,1.000000,10.136
7,wine.csv,123,3,0.972222,0.972369,0.974747,0.972222,10.121
8,wine.csv,123,4,0.971429,0.971300,0.973333,0.971429,10.144
9,wine.csv,123,5,1.000000,1.000000,1.000000,1.000000,10.124


CSV for wine: flaml_wine_task3_runtime_memory_metrics.csv


,dataset,seed,fold,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,wine.csv,42,1,10.163,0.739,7697.148,0.000278,31.9
1,wine.csv,42,2,10.155,0.796,7697.145,0.000306,31.1
2,wine.csv,42,3,10.123,0.674,7697.145,0.000362,33.9
3,wine.csv,42,4,10.145,0.765,7697.145,0.000345,35.0
4,wine.csv,42,5,10.148,0.710,7697.152,0.000401,34.1
5,wine.csv,123,1,10.154,0.616,7697.137,0.000361,34.1
6,wine.csv,123,2,10.136,0.672,7697.145,0.000362,37.3
7,wine.csv,123,3,10.121,0.745,7697.152,0.000362,33.1
8,wine.csv,123,4,10.144,0.750,7697.141,0.000315,28.8
9,wine.csv,123,5,10.124,0.685,7697.148,0.000402,32.0


In [4]:
# ---------------- Task 6 ----------------

from flaml import AutoML


def evaluate_pipeline_folds_times_flaml(leakage_free_final_output):
    for dataset_name, folds in leakage_free_final_output.items():
        dataset_base = dataset_name.replace('.csv', '')
        dataset_summary = []
        time_log_rows = []
        target_col = TARGET_COLS[dataset_name]

        for seed in RANDOM_SEEDS_PIPELINE:
            np.random.seed(seed)
            random.seed(seed)

            for fold_name, split in folds.items():
                df_train = split['train'].copy()
                df_val   = split['val'].copy()
                pipe_times = split['pipeline_time']
                pipe_times_copy = pipe_times.copy()
                pipe_times_copy["total_time"] = sum(pipe_times_copy.values())

                X_train = df_train.drop(columns=[target_col])
                y_train = df_train[target_col]
                X_val   = df_val.drop(columns=[target_col])
                y_val   = df_val[target_col]

                if X_train.shape[0] > 20000:
                    X_train_sample = X_train.sample(n=20000, random_state=42)
                    y_train_sample = y_train.loc[X_train_sample.index]
                else:
                    X_train_sample = X_train
                    y_train_sample = y_train

                temp_dir = tempfile.mkdtemp()
                start_time = time.time()

                automl = AutoML()
                automl.fit(
                    X_train=X_train_sample,
                    y_train=y_train_sample,
                    task=tasks_dict_classification_only[dataset_name],
                    time_budget=CV_TIME_BUDGET_PIPELINE,
                    eval_method='holdout',
                    ensemble=True,
                    estimator_list=["lgbm", "xgboost", "rf", "extra_tree"],
                    n_jobs=1,
                    seed=seed,
                    verbose=0
                )

                preds = automl.predict(X_val)
                if preds.ndim > 1:
                    preds = (preds[:, 0] > 0.5).astype(int)

                runtime = round(time.time() - start_time, 3)
                fold_idx = int(fold_name.split("_")[1])
                best_model = automl.best_estimator

                dataset_summary.append({
                    "dataset": dataset_name,
                    "seed": seed,
                    "fold": fold_idx,
                    "model_name": f"flaml - {best_model}",
                    "accuracy": accuracy_score(y_val, preds),
                    "f1": f1_score(y_val, preds, average='weighted'),
                    "precision": precision_score(y_val, preds, average='weighted'),
                    "recall": recall_score(y_val, preds, average='weighted'),
                    "runtime": runtime
                })

                row = {"dataset": dataset_name, "fold": fold_idx, "model_name": f"flaml - {best_model}"}
                row.update(pipe_times_copy)
                time_log_rows.append(row)

                temp_model_file = os.path.join(temp_dir, f"{dataset_base}_{seed}_fold{fold_idx}.pkl")
                with open(temp_model_file, "wb") as f:
                    pickle.dump(automl, f)
                del automl
                gc.collect()
                shutil.rmtree(temp_dir, ignore_errors=True)

        df_summary = pd.DataFrame(dataset_summary).drop_duplicates()
        df_times   = pd.DataFrame(time_log_rows).drop_duplicates()
        df_summary = df_summary.sort_values(["dataset", "seed", "fold"])
        df_times   = df_times.sort_values(["dataset", "fold"])

        df_summary.to_csv(os.path.join(PIPELINE_DATAFRAMES_METRICS_PATH, f"flaml_{dataset_base}_pipeline_summary.csv"), index=False)
        df_times.to_csv(os.path.join(PIPELINE_LOG_TIMES_PATH, f"flaml_{dataset_base}_pipeline_time_log.csv"), index=False)

evaluate_pipeline_folds_times_flaml(run_pipeline_on_folds_with_control(pipeline_data_dict))

In [5]:
for dataset_name in data_dict_classification_only.keys():
    dataset_base = dataset_name.replace('.csv', '')
    
    csv_file_1 = os.path.join(PIPELINE_DATAFRAMES_METRICS_PATH, f"flaml_{dataset_base}_pipeline_summary.csv")
    csv_file_2 = os.path.join(PIPELINE_LOG_TIMES_PATH, f"flaml_{dataset_base}_pipeline_time_log.csv")
    
    df1 = pd.read_csv(csv_file_1)
    df2 = pd.read_csv(csv_file_2)
    
    print(f"CSV for {dataset_base}: flaml_{dataset_base}_pipeline_summary.csv")
    display(df1) 
    print(f"CSV for {dataset_base}: flaml_{dataset_base}_pipeline_time_log.csv")
    display(df2) 



CSV for modeldata: flaml_modeldata_pipeline_summary.csv


,dataset,seed,fold,model_name,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,1,flaml - lgbm,0.918712,0.918558,0.918472,0.918712,42.886
1,modeldata.csv,42,2,flaml - rf,0.924474,0.924600,0.924782,0.924474,38.066
2,modeldata.csv,123,1,flaml - lgbm,0.918638,0.918548,0.918482,0.918638,36.448
3,modeldata.csv,123,2,flaml - lgbm,0.923487,0.923397,0.923334,0.923487,34.049


CSV for modeldata: flaml_modeldata_pipeline_time_log.csv


,dataset,fold,model_name,preprocessing,nan_guard_before_vif,vif,binner,nan_guard_before_poly,selector,total_time
0,modeldata.csv,1,flaml - lgbm,0.369047,0.139001,0.012078,0.047626,0.045500,0.021084,0.634336
1,modeldata.csv,2,flaml - rf,0.369983,0.137530,0.013085,0.085247,0.065731,0.022131,0.693707
2,modeldata.csv,2,flaml - lgbm,0.369983,0.137530,0.013085,0.085247,0.065731,0.022131,0.693707


CSV for titanic: flaml_titanic_pipeline_summary.csv


...

,dataset,fold,model_name,preprocessing,nan_guard_before_vif,vif,binner,nan_guard_before_poly,poly,selector,total_time
0,train.csv,1,flaml - xgboost,0.001002,0.0,0.001002,0.000000,0.0,0.0,0.0,0.002004
1,train.csv,2,flaml - xgboost,0.002005,0.0,0.000000,0.001002,0.0,0.0,0.0,0.003007


CSV for wine: flaml_wine_pipeline_summary.csv


,dataset,seed,fold,model_name,accuracy,f1,precision,recall,runtime
0,wine.csv,42,1,flaml - xgboost,0.932584,0.932042,0.932951,0.932584,31.184
1,wine.csv,42,2,flaml - extra_tree,0.887640,0.887279,0.896078,0.887640,31.221
2,wine.csv,123,1,flaml - extra_tree,0.932584,0.932299,0.934580,0.932584,31.153
3,wine.csv,123,2,flaml - extra_tree,0.887640,0.887279,0.896078,0.887640,31.246


CSV for wine: flaml_wine_pipeline_time_log.csv


,dataset,fold,model_name,preprocessing,nan_guard_before_vif,vif,binner,nan_guard_before_poly,poly,selector,total_time
0,wine.csv,1,flaml - xgboost,0.001147,0.0,0.001002,0.000000,0.0,0.001002,0.0,0.003151
1,wine.csv,1,flaml - extra_tree,0.001147,0.0,0.001002,0.000000,0.0,0.001002,0.0,0.003151
2,wine.csv,2,flaml - extra_tree,0.002068,0.0,0.000000,0.001003,0.0,0.000000,0.0,0.003071
